In [ ]:
# 범죄 데이터 읽기
# 컬럼명 변경 : 기존컬럼 -> ["자치구", "발생합계", "검거합계", "살인발생", "살인검거", 
#                           "강도발생", "강도검거", "강간발생", "강간검거", 
#                           "절도발생", "절도검거", "폭력발생", "폭력검거"]
# 자치구 컬럼을 index로 변경
# (살인, 강도, 강간, 절도, 폭력) 검거율 컬럼 만들기
# ~검거 컬럼 제거
# 소계 행 제거
# ~발생 -> ~ 으로 변경 (발생 단어 제거 : 살인발생 -> 살인)
# 각 범죄가 가장 많은 자치구 확인
# 검거율 높은 자치구 확인
# 각 컬럼의 값을 0 ~ 1 사이의 값으로 변경 ( scale 변경 )
# 파일 저장 : processed-seoul-crime.csv